1. conda install all strox dependencies
2. use jupyter not colab 
3. add conda env package list to commit
4. ok imports

In [230]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import xml.etree.ElementTree as ET
from lxml import etree
import warnings

In [231]:
import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from scipy.sparse import hstack

In [232]:
home = '/opt/anaconda3/stanford-postagger-full-2020-08-06'
from nltk.tag.stanford import StanfordPOSTagger as POS_Tag
from nltk import word_tokenize
_path_to_model = home + '/models/english-bidirectional-distsim.tagger' 
_path_to_jar = home + '/stanford-postagger.jar'
stanford_tag = POS_Tag(model_filename=_path_to_model, path_to_jar=_path_to_jar)

In [233]:
path_train = 'ABSA16_Laptops_Train_English_SB2.xml'
path_test = 'EN_LAPT_SB2_TEST.xml.A'

so [joblib](https://joblib.readthedocs.io/en/latest/index.html) is python pipeline processing lib

sklearn is used here. this is a maintained and reliable library according to [this guy](https://www.youtube.com/watch?v=rvVkVsG49uU)

- get 
    - ABSA16_Laptops_Train_English_SB2.xml 
    - EN_LAPT_SB2_TEST.xml
    - stanford pos tagger stuff
    - tagged_text_list_train.pkl files
- add function definitions

In [234]:
def get_list(path):
    tree=ET.parse(path)
    root = tree.getroot()
    text_list = []
    opinion_list = []
    for review in root.findall('Review'):
        text_string=""
        opinion_inner_list=[]
        for sent in review.findall('./sentences/sentence'):
            text_string= text_string+ " "+ sent.find('text').text
        text_list.append(text_string)
        for opinion in review.findall('./Opinions/Opinion'):
            opinion_dict = {
                opinion.get('category').replace('#','_'): opinion.get('polarity')
            }
            opinion_inner_list.append(opinion_dict)
        opinion_list.append(opinion_inner_list)
    return text_list,opinion_list

In [235]:
def get_most_common_aspect(opinion_list):
    import nltk
    opinion= []
    for inner_list in opinion_list:
        for _dict in inner_list:
            for key in _dict:
                opinion.append(key)
    most_common_aspect = [k for k,v in nltk.FreqDist(opinion).most_common(20)]
    return most_common_aspect

In [236]:
def get_data_frame(text_list,opinion_list,most_common_aspect):
    data={'Review':text_list}
    df = pd.DataFrame(data)
    if opinion_list:
        for inner_list in opinion_list:
            for _dict in inner_list:
                for key in _dict:
                    if key in most_common_aspect:
                        df.loc[opinion_list.index(inner_list),key]=_dict[key]
    return df

In [254]:
def get_aspect_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect]=df[common_aspect].replace(['positive','negative','neutral','conflict'],[1,1,1,1])
    df = df.fillna(0)
    return df

In [238]:
def get_positive_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect]=df[common_aspect].replace(['positive'],[1])
        df[common_aspect]=df[common_aspect].replace(['negative','neutral','conflict'],[0,0,0])
    df = df.fillna(0)
    return df

In [239]:
def get_negative_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect]=df[common_aspect].replace(['negative'],[1])
        df[common_aspect]=df[common_aspect].replace(['positive','neutral','conflict'],[0,0,0])
    df = df.fillna(0)
    return df

In [240]:
def get_neutral_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect]=df[common_aspect].replace(['neutral','conflict'],[1,1])
        df[common_aspect]=df[common_aspect].replace(['negative','positive'],[0,0])
    df = df.fillna(0)
    return df

In [241]:
def posTag(review):
    tagged_text_list=[]
    for text in review:
        tagged_text_list.append(stanford_tag.tag(word_tokenize(text)))
    return tagged_text_list

In [242]:
def filterTag(tagged_review):
    final_text_list=[]
    for text_list in tagged_review:
        final_text=[]
        for word,tag in text_list:
            if tag in ['NN','NNS','NNP','NNPS','RB','RBR','RBS','JJ','JJR','JJS','VB','VBD','VBG','VBN','VBP','VBZ']:
                final_text.append(word)
        final_text_list.append(' '.join(final_text))
    return final_text_list

In [243]:
def get_dict_aspect(y,most_common_aspect):
    position=[]
    for innerlist in y:
        position.append([i for i, j in enumerate(innerlist) if j == 1])
    sorted_common=sorted(most_common_aspect)
    dict_aspect=[]
    for innerlist in position:
        inner_dict={}
        for word in sorted_common:
            if sorted_common.index(word) in innerlist:
                inner_dict[word]= 5
            else:
                inner_dict[word]=0
        dict_aspect.append(inner_dict)
    return dict_aspect

In [244]:
def get_sample(l1,l2=None,n=1):
    for i in range(0,n):
        rand_int = int(np.random.rand() * len(l1))
        if l2:
            print(l1[rand_int]), '\n\n', print(l2[rand_int])
        else:
            print(l1[rand_int])
            return l1[rand_int]

In [245]:
def get_meta(l):
    print('type: ',type(l))
    print('len: ',len(l))

**main control flow in [thestrox Github code](https://github.com/thestrox/Aspect-Based-Sentiment-Analysis/blob/master/Aspect%20Based%20Sentiment%20Analysis.ipynb)**

In [246]:
train_text_list, train_opinion_list = get_list(path_train)
# get_sample(train_text_list,train_opinion_list,n=2)

In [247]:
most_common_aspect = get_most_common_aspect(train_opinion_list)
# most_common_aspect

In [248]:
tagged_text_list_train=joblib.load('tagged_text_list_train.pkl')

In [249]:
final_train_text_list = filterTag(tagged_text_list_train)
# get_sample(tagged_text_list_train,final_train_text_list)

Each train review and its SB2 (aspect,sentiment) pair is POS-tagged, and only nouns, adjectives, verbs and adverbs are retained according to [Penn Treebank](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) POS table.

In [250]:
df_train = get_data_frame(train_text_list,train_opinion_list,most_common_aspect)
# df_train.sample(3)
# df_train.describe()
df_train.columns

Index(['Review', 'LAPTOP_GENERAL', 'LAPTOP_OPERATION_PERFORMANCE',
       'LAPTOP_QUALITY', 'DISPLAY_QUALITY', 'LAPTOP_DESIGN_FEATURES',
       'LAPTOP_USABILITY', 'LAPTOP_PRICE', 'COMPANY_GENERAL',
       'SUPPORT_QUALITY', 'LAPTOP_CONNECTIVITY', 'LAPTOP_PORTABILITY',
       'BATTERY_OPERATION_PERFORMANCE', 'MULTIMEDIA_DEVICES_QUALITY',
       'KEYBOARD_DESIGN_FEATURES', 'DISPLAY_GENERAL', 'LAPTOP_MISCELLANEOUS',
       'SOFTWARE_GENERAL', 'OS_GENERAL', 'DISPLAY_DESIGN_FEATURES',
       'OS_USABILITY'],
      dtype='object')

In [255]:
df_train_aspect = get_aspect_data_frame(df_train,most_common_aspect)